In [1]:
using TensorKit
import TensorKit.×
using KrylovKit
include("./iPEPS/iPEPS.jl")
include("./iPEPS/swap_gate.jl")
include("./iPEPS/util.jl")

inv_sqrt

In [ ]:
## follow convention (l, t, σ, r, b)
## domain: r, b
## codomain: l, t, σ
M11 = TensorMap(randn, ComplexF64, ℂ^3 ⊗ ℂ^3 ⊗ ℂ^2, ℂ^3 ⊗ ℂ^3)
@show domain(M11)
@show space(M11)
M12 = TensorMap(randn, ComplexF64, ℂ^3 ⊗ ℂ^3 ⊗ ℂ^2, ℂ^3 ⊗ ℂ^3)

## contraction 后用; 分割 (codomain; domain), 原来的domain 放到 codomain，或反过来，空间都要变为对偶空间（加'）
@tensor M1112[l1, t1, σ1, σ2, b1; t2, r2, b2] := M11[l1, t1, σ1, r1, b1] * M12[r1, t2, σ2, r2, b2]
# B = permute(A, (3,), (1, 2, 4, 5))
# @show domain(B)
@show domain(M1112)
@show codomain(M1112)

In [2]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 1, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((1, 1//2) => 1, (0, 1) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet
A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
Ms = [A, A, A, A]
ipeps = iPEPS(Ms, 2, 2)
Abar = bar(A)
@show space(Abar)
gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
@show space(gate1)
gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
@show space(gate2)
gate3 = swap_gate(space(A)[4], space(A)[5])
@show space(gate3)
gate4 = swap_gate(space(A)[1], space(A)[2])
@show space(gate4)
gate5 = swap_gate(space(A)[1], space(Abar)[2])
@show space(gate5)

Initializing iPEPS with eltype ComplexF64

space(Abar) = (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1)' ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)' ⊗ Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)') ← (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1)' ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)')
space(gate1) = (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)) ← (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1))
space(gate2) = (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1)' ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)') ← (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1)' ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)')
space(gate3) = (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1)' ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)') ← (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1)' ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)')
space(gate4) = (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)) ← (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1))
space(gate5) = (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1) ⊗ Rep[ℤ₂ 

(Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)') ← (Rep[ℤ₂ × SU₂]((0, 0)=>1, (1, 1/2)=>1) ⊗ Rep[ℤ₂ × SU₂]((1, 1/2)=>1, (0, 1)=>1)')

In [5]:
A = id(pspace)
@show A
exp!(A)
@show A
nothing

A = TensorMap(Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1) ← Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)):
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()):
 1.0  0.0
 0.0  1.0
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()):
 1.0

A = TensorMap(Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1) ← Rep[ℤ₂ × SU₂]((0, 0)=>2, (1, 1/2)=>1)):
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((0, 0),), (0, 0), (false,), ()):
 2.718281828459045  0.0
 0.0                2.718281828459045
* Data for fusiontree FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()) ← FusionTree{Irrep[ℤ₂ × SU₂]}(((1, 1/2),), (1, 1/2), (false,), ()):
 2.718281828459045



In [ ]:
include("./simple_update/simple_update.jl")
ipepsΓΛ = iPEPSΓΛ(ipeps)
nothing

In [ ]:
@show rank(ipepsΓΛ[1,1].Γ)
@show rank(ipepsΓΛ[1,1].l)
@assert ipepsΓΛ[1,1].l == ipepsΓΛ[2,1].r
@show space(ipepsΓΛ[1,1].Γ)[1]
@show space(ipepsΓΛ[1,1].Γ)[2]
@show space(ipepsΓΛ[1,1].Γ)[3]
@show space(ipepsΓΛ[1,1].Γ)[4]
@show space(ipepsΓΛ[1,1].Γ)[5]
nothing